In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import os
os.chdir("/home/furfa/work/ai-academy2019/notebooks")
pd.set_option("max_columns", 10000)

%pylab inline
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

import json
from IPython import display

from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


In [2]:
base_dir = ".."
input_dir = os.path.join(base_dir, "input")
processed_dir = os.path.join(input_dir, "processed")
os.listdir( input_dir )

['dota2_abilities.csv',
 'dota2_skill_train.csv',
 'dota2_skill_test.csv',
 'ai-academy-2019-skill-prediction-data-csv-v1.zip',
 'dota2_skill_test.jsonlines',
 'processed',
 'dota2_items.csv',
 'dota2_heroes.csv',
 'ai-academy-2019-skill-prediction-data-jsonlines.zip',
 'dota2_skill_train.jsonlines']

In [3]:
train_main = pd.read_csv( os.path.join(input_dir, 'dota2_skill_train.csv'), index_col=0 )
test_main = pd.read_csv( os.path.join(input_dir, 'dota2_skill_test.csv'), index_col=0 )

In [4]:
train_proc = pd.read_csv( os.path.join(processed_dir, "train_proc.csv") )
test_proc = pd.read_csv( os.path.join(processed_dir, "test_proc.csv") )

In [33]:
radiant_win = pd.concat( (train_main, test_main), axis=0).winner_team == "radiant"

In [34]:
train_main.shape[0] + test_main.shape[0]

143136

In [6]:
columns_selection = [f"{base_name}_hero_{num}" for num in range(0,5) for base_name in ("dire", "radiant")]
train_team = train_proc[columns_selection]
test_team = test_proc[columns_selection]

In [7]:
np.unique(train_team.values.reshape(-1))

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 119, 120, 121, 129])

In [8]:
ntrain = train_team.shape[0]
ntest = test_team.shape[0]
all_data_team = pd.concat((train_team, test_team), axis=0)

In [9]:
def team_encode(df, team):
    df_size = df.shape[0]
    enc = np.zeros((df_size, 129))
    columns_selection = [f"{team}_hero_{num}" for num in range(0,5)]
    sel_team = df[columns_selection].values
    for i in tqdm(range(df_size)):
        for hero in range(1, 129+1):
            if hero in sel_team[i]:
                enc[i, hero-1] += 1
    
    return enc

In [10]:
data = np.hstack( 
    ( 
        team_encode(all_data_team, "dire"), 
        team_encode(all_data_team, "radiant") 
    )
)

100%|██████████| 143136/143136 [01:32<00:00, 1552.74it/s]


In [15]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier

In [12]:
cross_val_score(
    KNeighborsClassifier(), 
    data, 
    radiant_win, 
    cv=6,
    n_jobs=-1,
    verbose=1
)

[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 44.0min finished


array([0.419056  , 0.42672703, 0.42538565, 0.41679242, 0.41964286,
       0.42211603])

In [35]:
pred = cross_val_predict(
    KNeighborsClassifier(), 
    data, 
    radiant_win, 
    cv=6,
    n_jobs=-1,
    verbose=2,
    method='predict_proba'
)

[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed: 46.9min remaining: 46.9min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 47.1min finished


array([[0.8, 0.2],
       [0.6, 0.4],
       [0.2, 0.8],
       ...,
       [0.2, 0.8],
       [0.4, 0.6],
       [0.8, 0.2]])

In [43]:
pred = pd.DataFrame(pred[:,0], columns=["radiant_win_%"])
train = pred[:ntrain]
train.index = train_main.index
test = pred[ntrain:]
test.index = test_main.index

In [44]:
try:
    processed_dir = os.path.join(input_dir, "processed")
    os.mkdir(processed_dir)
except:
    pass
finally:
    train.to_csv(
        os.path.join(processed_dir, "train_team.csv")
    )
    test.to_csv(
        os.path.join(processed_dir, "test_team.csv")
    )